# SQL Exploratory Data Analysis on SpaceX Launch Data

This notebook performs exploratory data analysis (EDA) using SQL on historical SpaceX Falcon 9 launch data stored in a SQLite database.  
The objective is to extract insights related to launch outcomes, payloads, launch sites, and mission characteristics to support downstream analysis and modeling.


## Introduction

In this notebook, SQL is used within a Jupyter environment to explore and analyze SpaceX Falcon 9 launch data stored in a SQLite database.  
The analysis focuses on understanding launch patterns, success rates, payload characteristics, and launch site behavior using structured queries.


## Overview of the Dataset

SpaceX has achieved significant cost reductions in spaceflight by reusing the first stage of the Falcon 9 rocket.  
Determining whether the first stage will successfully land is therefore a key factor in estimating launch costs.

This dataset contains records of SpaceX Falcon 9 launches, including information about launch sites, payload mass, orbit type, booster landing outcomes, and mission details.  
Each row represents a single payload carried during a SpaceX mission.


## Database Connection

A SQLite database is used to store the SpaceX launch data.  
The database connection is established below, allowing SQL queries to be executed directly within the Jupyter notebook.


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
import pandas as pd
df = pd.read_csv('Spacex.csv')
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [6]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## SQL-Based Exploratory Analysis

The following sections use SQL queries to explore launch sites, payload characteristics, and mission outcomes from the SpaceX launch dataset.
### Unique Launch Sites

This query retrieves the distinct launch sites used in SpaceX missions.


In [8]:
%sql SELECT DISTINCT Launch_Site FROM  SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### Launch Sites Starting with 'CCA'

This query filters launch records where the launch site name begins with 'CCA'.


In [9]:
%sql SELECT * FROM SPACEXTABLE WHERE Launch_Site LIKE 'CCA%' LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Total Payload Mass for NASA (CRS) Missions

This query calculates the total payload mass carried by Falcon 9 boosters for missions conducted under NASA’s Commercial Resupply Services (CRS) program.


In [10]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) AS  total_payload_mass  FROM SPACEXTABLE WHERE Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


total_payload_mass
45596


**Observation:**  
The total payload mass indicates the overall cargo capacity delivered by SpaceX for NASA CRS missions across all recorded launches.


### Average Payload Mass for Falcon 9 v1.1

This query computes the average payload mass carried by missions using the Falcon 9 v1.1 booster version.


In [11]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) AS  average_payload FROM SPACEXTABLE WHERE Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


average_payload
2928.4


**Observation:**  
The average payload mass reflects the typical lifting capability of the Falcon 9 v1.1 booster configuration during its operational period.


### First Successful Ground Pad Landing

This query identifies the earliest date on which a Falcon 9 booster successfully landed on a ground pad.


In [12]:
%%sql SELECT MIN(Date) AS DATE
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


DATE
2015-12-22


**Observation:**  
The result highlights a key milestone in SpaceX’s reusability program, marking the first successful ground pad landing of a Falcon 9 booster.


### Successful Drone Ship Landings with Medium Payload Mass

This query retrieves booster versions that successfully landed on a drone ship while carrying payloads between 4000 kg and 6000 kg.


In [13]:
%sql SELECT Booster_Version FROM SPACEXTABLE WHERE Landing_Outcome LIKE 'Success%drone ship%' AND PAYLOAD_MASS__KG_> 4000	AND PAYLOAD_MASS__KG_ < 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


**Observation:**  
The listed booster versions demonstrate reliable drone ship landing performance for medium-range payload missions.


### Mission Outcome Distribution

This query calculates the total number of successful and failed mission outcomes across all recorded SpaceX launches.


In [14]:
%%sql SELECT CASE
         WHEN Mission_Outcome LIKE 'Success%' THEN 'Success'
         ELSE 'Failure'
       END AS Mission_Result,
       COUNT(*) AS total_number
FROM SPACEXTABLE
GROUP BY Mission_Result

 * sqlite:///my_data1.db
Done.


Mission_Result,total_number
Failure,1
Success,100


**Observation:**  
The results show a strong dominance of successful missions, highlighting SpaceX’s high mission reliability over time.


### Booster Versions with Maximum Payload Capacity

This query identifies booster versions that have carried the maximum recorded payload mass using a subquery and aggregate function.


In [15]:
%sql SELECT STRING_AGG(Booster_Version, ',') AS booster_versions  FROM (SELECT DISTINCT Booster_Version FROM SPACEXTABLE WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE)) AS NEW_TABLE

 * sqlite:///my_data1.db
Done.


booster_versions
"F9 B5 B1048.4,F9 B5 B1049.4,F9 B5 B1051.3,F9 B5 B1056.4,F9 B5 B1048.5,F9 B5 B1051.4,F9 B5 B1049.5,F9 B5 B1060.2 ,F9 B5 B1058.3 ,F9 B5 B1051.6,F9 B5 B1060.3,F9 B5 B1049.7"


**Observation:**  
Multiple booster versions share the maximum payload capacity, indicating consistent performance across different Falcon 9 boosters.


### Drone Ship Landing Failures in 2015

This query extracts records of drone ship landing failures in the year 2015, including the launch month, booster version, and launch site.


In [16]:
%%sql SELECT 
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE 'Failure%drone ship%'
  AND substr(Date, 1, 4) = '2015'

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


**Observation:**  
The results indicate that drone ship landing failures in 2015 were limited to specific months and booster configurations.


### Ranking of Landing Outcomes by Frequency

This query ranks different landing outcomes based on their frequency between June 2010 and March 2017 using a window function.


In [17]:
 %sql SELECT Landing_Outcome,COUNT_ALL,ROW_NUMBER() OVER(ORDER BY COUNT_ALL DESC) AS RANK FROM (SELECT Landing_Outcome,COUNT(*) AS COUNT_ALL FROM SPACEXTABLE WHERE Date >=' 2010-06-04' AND Date <= '2017-03-20' GROUP BY Landing_Outcome) AS NEW_TABLE


 * sqlite:///my_data1.db
Done.


Landing_Outcome,COUNT_ALL,RANK
No attempt,10,1
Failure (drone ship),5,2
Success (drone ship),5,3
Controlled (ocean),3,4
Success (ground pad),3,5
Failure (parachute),2,6
Uncontrolled (ocean),2,7
Precluded (drone ship),1,8


**Observation:**  
The ranking shows that non-attempted and drone ship-related outcomes were the most frequent, reflecting the evolution of SpaceX’s landing strategies over time.


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->
